# Notebook used to apply standard compression methods to Kodak dataset image for evaluation
## Importing all the necessary packages and initializing input and output directories

In [1]:
import os
import ntpath
from PIL import Image
from pathlib import Path
import glob
import timeit
import subprocess
from numpy import mean
from datetime import datetime

vvenc_path = Path('/home/filippo/Downloads/vvenc-1.9.1/install/bin/vvencapp').resolve()
print ("VVEncAPP Path: ", str(vvenc_path))
vvdec_path = Path('/home/filippo/Downloads/vvdec-2.1.2/install/bin/vvdecapp').resolve()
print ("VVDevAPP Path: ", str(vvdec_path))

current_path = Path('.').resolve()
print("Current path: ",str(current_path))
source_path = str(current_path)+'/IMAGES/PNG_IMAGES'
source_path.replace(os.sep,ntpath.sep)
print("Source: ",source_path)

VVEncAPP Path:  /home/filippo/Downloads/vvenc-1.9.1/install/bin/vvencapp
VVDevAPP Path:  /home/filippo/Downloads/vvdec-2.1.2/install/bin/vvdecapp
Current path:  /home/filippo/DataspellProjects/ImageCompressionAI
Source:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/PNG_IMAGES


# JPEG conversion
## Setting output dir for JPEG file conversion

In [43]:
destination_path = str(current_path)+'/IMAGES/JPEG_IMAGES'
destination_path.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

destination_path_JPEG = str(current_path)+'/IMAGES/JPEG_IMAGES/JPEG'
destination_path_JPEG.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path_JPEG)
if os.path.isdir(destination_path_JPEG) & os.path.exists(destination_path_JPEG):
    try:
        files = os.listdir(destination_path_JPEG)
        for file in files:
            file_path = os.path.join(destination_path_JPEG, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path_JPEG)

Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG_IMAGES
Destination directory has been emptied
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG_IMAGES/JPEG
Destination directory has been emptied


## Converting images into JPEG format

In [46]:
compression_times = []
quality_index = 3 # The image quality, on a scale from 0 (worst) to 95 (best)
# 3 = 0.174bpp | 
for file in glob.glob(str(source_path)+'/*.png'):
    image = Image.open(file)
    image = image.convert('RGB')
    starttime = timeit.default_timer()
    image.save(str(destination_path_JPEG)+'/'+Path(file).stem+'.jpeg','jpeg',subsampling=1, quality=quality_index)
    endtime = timeit.default_timer()
    execution_time = endtime-starttime
    compression_times.append(execution_time)
    print('Encoded',Path(file).stem,' | Execution time: ',str(execution_time),' seconds')
    image.close()

Encoded IMG0001  | Execution time:  0.014397283000107564  seconds
Encoded IMG0002  | Execution time:  0.014051073000018732  seconds
Encoded IMG0003  | Execution time:  0.0139553110000179  seconds
Encoded IMG0004  | Execution time:  0.014140986000029443  seconds
Encoded IMG0005  | Execution time:  0.01458688799993979  seconds
Encoded IMG0006  | Execution time:  0.014183227000103216  seconds
Encoded IMG0007  | Execution time:  0.014024931999983892  seconds
Encoded IMG0008  | Execution time:  0.014490074999912395  seconds
Encoded IMG0009  | Execution time:  0.014391772999942987  seconds
Encoded IMG0010  | Execution time:  0.014129056000001583  seconds
Encoded IMG0011  | Execution time:  0.014196948000062548  seconds
Encoded IMG0012  | Execution time:  0.013726245000043491  seconds
Encoded IMG0013  | Execution time:  0.01478362299985747  seconds
Encoded IMG0014  | Execution time:  0.014305639999975028  seconds
Encoded IMG0015  | Execution time:  0.014180567000039446  seconds
Encoded IMG001

## Converting JPEG compressed images back to PNG for later comparison

In [48]:
for file in glob.glob(str(destination_path_JPEG)+'/*.jpeg'):
    image = Image.open(file)
    image = image.convert('RGB')
    image.save(str(destination_path)+'/'+Path(file).stem+'.png','png')
    print(Path(file).stem)
    image.close()

IMG0001
IMG0002
IMG0003
IMG0004
IMG0005
IMG0006
IMG0007
IMG0008
IMG0009
IMG0010
IMG0011
IMG0012
IMG0013
IMG0014
IMG0015
IMG0016
IMG0017
IMG0018
IMG0019
IMG0020
IMG0021
IMG0022
IMG0023
IMG0024


## Compute bits per pixel of converted images

In [47]:
bpp_list=[]
count = 0
log = open(str(destination_path)+'/log.txt', 'w')
log.writelines([str(datetime.now()),'\n'])
for file in glob.glob(str(destination_path_JPEG)+'/*.jpeg'):
    image = Image.open(file)
    file_size = os.path.getsize(file) * 8
    pixels = image.width * image.height
    bits_per_pixel = file_size/pixels
    bpp_list.append(bits_per_pixel)
    log_string = 'Image:' + str(Path(file).stem) + '.jpeg\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(file_size) + '\tWidth:' + str(image.width) + '\tHeight:' + str(image.height) + '\tPixels:' + str(pixels) + '\tCompressionTime:' + str(compression_times[count])
    log.writelines([log_string,'\n'])
    print(log_string)
    image.close()
    count+=1
    
log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')])    
print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')
log.close()

Image:IMG0001.jpeg	bpp:0.18744659423828125	Size:1179312	Width:3072	Height:2048	Pixels:6291456	CompressionTime:0.014397283000107564
Image:IMG0002.jpeg	bpp:0.16878255208333334	Size:1061888	Width:3072	Height:2048	Pixels:6291456	CompressionTime:0.014051073000018732
Image:IMG0003.jpeg	bpp:0.16411590576171875	Size:1032528	Width:3072	Height:2048	Pixels:6291456	CompressionTime:0.0139553110000179
Image:IMG0004.jpeg	bpp:0.17052841186523438	Size:1072872	Width:2048	Height:3072	Pixels:6291456	CompressionTime:0.014140986000029443
Image:IMG0005.jpeg	bpp:0.1914393107096354	Size:1204432	Width:3072	Height:2048	Pixels:6291456	CompressionTime:0.01458688799993979
Image:IMG0006.jpeg	bpp:0.1758855183919271	Size:1106576	Width:3072	Height:2048	Pixels:6291456	CompressionTime:0.014183227000103216
Image:IMG0007.jpeg	bpp:0.17060089111328125	Size:1073328	Width:3072	Height:2048	Pixels:6291456	CompressionTime:0.014024931999983892
Image:IMG0008.jpeg	bpp:0.20340474446614584	Size:1279712	Width:3072	Height:2048	Pixels:62

# JPEG2000 Conversion
## Setting output directory for JPEG2000 file conversion

In [2]:
destination_path = str(current_path)+'/IMAGES/JPEG2000_IMAGES'
destination_path.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

destination_path_JPEG2000 = str(current_path)+'/IMAGES/JPEG2000_IMAGES/JPEG2000'
destination_path_JPEG2000.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path_JPEG2000)
if os.path.isdir(destination_path_JPEG2000) & os.path.exists(destination_path_JPEG2000):
    try:
        files = os.listdir(destination_path_JPEG2000)
        for file in files:
            file_path = os.path.join(destination_path_JPEG2000, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path_JPEG2000)

Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
Destination directory has been emptied
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES/JPEG2000
Destination directory has been emptied


## Converting Images to JPEG2000

In [41]:
compression_times = []
quality_layers = [134]
# 140 = 0.171bpp
for file in glob.glob(str(source_path)+'/*.png'):
    image = Image.open(file)
    image = image.convert('RGB')
    starttime = timeit.default_timer()
    image.save(str(destination_path_JPEG2000)+'/'+Path(file).stem+'.jp2','jpeg2000',irreversible=True, quality_mode='rates',quality_layers=quality_layers)
    endtime = timeit.default_timer()
    execution_time = endtime-starttime
    compression_times.append(execution_time)
    print('Encoded',Path(file).stem,' | Execution time: ',execution_time,' seconds')
    image.close()

Encoded IMG0001  | Execution time:  2.7191758779999873  seconds
Encoded IMG0002  | Execution time:  2.3042827149999994  seconds
Encoded IMG0003  | Execution time:  2.1661367789999986  seconds
Encoded IMG0004  | Execution time:  2.2489404059999742  seconds
Encoded IMG0005  | Execution time:  2.3272354890000315  seconds
Encoded IMG0006  | Execution time:  2.2892192189999605  seconds
Encoded IMG0007  | Execution time:  2.4990200739998727  seconds
Encoded IMG0008  | Execution time:  3.6591254510001363  seconds
Encoded IMG0009  | Execution time:  2.93096153700003  seconds
Encoded IMG0010  | Execution time:  2.8362841910000043  seconds
Encoded IMG0011  | Execution time:  2.4757267610000326  seconds
Encoded IMG0012  | Execution time:  2.071358569999802  seconds
Encoded IMG0013  | Execution time:  2.8865670499999396  seconds
Encoded IMG0014  | Execution time:  2.368207739999889  seconds
Encoded IMG0015  | Execution time:  1.9898902400000225  seconds
Encoded IMG0016  | Execution time:  2.301266

## Converting JPEG200 compressed images back to PNG for later comparison

In [9]:
for file in glob.glob(str(destination_path_JPEG2000)+'/*.jp2'):
    image = Image.open(file)
    image = image.convert('RGB')
    image.save(str(destination_path)+'/'+Path(file).stem+'.png','png')
    print(Path(file).stem)
    image.close()

IMG0001
IMG0002
IMG0003
IMG0004
IMG0005
IMG0006
IMG0007
IMG0008
IMG0009
IMG0010
IMG0011
IMG0012
IMG0013
IMG0014
IMG0015
IMG0016
IMG0017
IMG0018
IMG0019
IMG0020
IMG0021
IMG0022
IMG0023
IMG0024


## Compute bits per pixel of converted images

In [42]:
bpp_list=[]
count = 0
log = open(str(destination_path)+'/log.txt', 'w')
log.writelines([str(datetime.now()),'\n'])
for file in glob.glob(str(destination_path_JPEG2000)+'/*.jp2'):
    image = Image.open(file)
    file_size = os.path.getsize(file) * 8
    pixels = image.width * image.height
    bits_per_pixel = file_size/pixels
    bpp_list.append(bits_per_pixel)
    log_string = 'Image:' + str(Path(file).stem) + '.jp2\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(file_size) + '\tWidth:' + str(image.width) + '\tHeight:' + str(image.height) + '\tPixels:' + str(pixels) + '\tCompressionTime:' + str(compression_times[count])
    log.writelines([log_string,'\n'])
    print(log_string)
    image.close()
    count +=1

log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')])
print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')
log.close()

Image:IMG0001.jp2	bpp:0.17907587687174478	Size:1126648	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.7191758779999873
Image:IMG0002.jp2	bpp:0.1788762410481771	Size:1125392	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.3042827149999994
Image:IMG0003.jp2	bpp:0.17910003662109375	Size:1126800	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.1661367789999986
Image:IMG0004.jp2	bpp:0.17908859252929688	Size:1126728	Width:2048	Height:3072	Pixels:6291456	CompressionTime:2.2489404059999742
Image:IMG0005.jp2	bpp:0.17909367879231772	Size:1126760	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.3272354890000315
Image:IMG0006.jp2	bpp:0.17896652221679688	Size:1125960	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.2892192189999605
Image:IMG0007.jp2	bpp:0.17886734008789062	Size:1125336	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.4990200739998727
Image:IMG0008.jp2	bpp:0.1790339152018229	Size:1126384	Width:3072	Height:2048	Pixels:6291456	CompressionTi

# BPG Conversion
## Setting output directory for BPG conversion

In [10]:
destination_path = str(current_path)+'/IMAGES/BPG_IMAGES'
destination_path.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

destination_path_BPG = str(current_path)+'/IMAGES/BPG_IMAGES/BPG'
destination_path_BPG.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path_BPG)
if os.path.isdir(destination_path_BPG) & os.path.exists(destination_path_BPG):
    try:
        files = os.listdir(destination_path_BPG)
        for file in files:
            file_path = os.path.join(destination_path_BPG, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path_BPG)

Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/BPG_IMAGES
Destination directory has been emptied
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/BPG_IMAGES/BPG
Destination directory has been emptied


## Converting Images to BPG

In [32]:
execution_times = []
quantizer_parameter = 42 #set quantizer parameter (smaller gives better quality,range: 0-51, default = 29)
# 42 = 0.179 bpp
for file in os.listdir(source_path):
    source_file = str(source_path + '/' + file)
    source_file.replace(os.sep,ntpath.sep)
    destination_file = str(destination_path_BPG + '/' + str(Path(file).stem) + '.bpg')
    destination_file.replace(os.sep,ntpath.sep)
    starttime = timeit.default_timer()
    subprocess.run(['bpgenc','-c','rgb','-f','420','-m','1','-q',str(quantizer_parameter),'-o',str(destination_file),str(source_file)])
    endtime = timeit.default_timer()
    execution_time = endtime - starttime
    execution_times.append(execution_time)
    print('Encoded', Path(file).stem, ' | Execution time: ', str(execution_time), ' seconds')

Encoded IMG0001  | Execution time:  1.037423851000085  seconds
Encoded IMG0002  | Execution time:  0.5855450019998898  seconds
Encoded IMG0003  | Execution time:  0.49724511199997323  seconds
Encoded IMG0004  | Execution time:  0.5180449220001719  seconds
Encoded IMG0005  | Execution time:  0.5686324610001066  seconds
Encoded IMG0006  | Execution time:  0.5500194319999991  seconds
Encoded IMG0007  | Execution time:  0.5140891809999175  seconds
Encoded IMG0008  | Execution time:  0.5938189830001193  seconds
Encoded IMG0009  | Execution time:  0.473930682999935  seconds
Encoded IMG0010  | Execution time:  0.4858221339998181  seconds
Encoded IMG0011  | Execution time:  0.5422173450001537  seconds
Encoded IMG0012  | Execution time:  0.49144841999986966  seconds
Encoded IMG0013  | Execution time:  0.6254992750000383  seconds
Encoded IMG0014  | Execution time:  0.5558055129999957  seconds
Encoded IMG0015  | Execution time:  0.5134042619999946  seconds
Encoded IMG0016  | Execution time:  0.51

## Converting BPG compressed images back to PNG for later comparison

In [34]:
for file in os.listdir(destination_path_BPG):
    source_file = str(destination_path_BPG + '/' + file)
    source_file.replace(os.sep,ntpath.sep)
    destination_file = str(destination_path + '/' + str(Path(file).stem) + '.png')
    destination_file.replace(os.sep,ntpath.sep)
    subprocess.run(['bpgdec','-o',str(destination_file),str(source_file)])
    print(Path(file).stem)

IMG0001
IMG0002
IMG0003
IMG0004
IMG0005
IMG0006
IMG0007
IMG0008
IMG0009
IMG0010
IMG0011
IMG0012
IMG0013
IMG0014
IMG0015
IMG0016
IMG0017
IMG0018
IMG0019
IMG0020
IMG0021
IMG0022
IMG0023
IMG0024


## Compute bits per pixel of converted images

In [33]:
bpp_list=[]
count = 0
log = open(str(destination_path)+'/log.txt', 'w')
log.writelines([str(datetime.now()),'\n'])
for file in os.listdir(destination_path_BPG):
    filepath = str(destination_path_BPG) + '/' + str(file)
    filepath.replace(os.sep,ntpath.sep)
    file_size = os.path.getsize(filepath) * 8
    output = subprocess.Popen(['bpgdec','-i',str(filepath)],stdout=subprocess.PIPE).communicate()
    dimensions = (((str(output).split(' '))[0].split('='))[1].split('x'))
    pixels = int(dimensions[0]) * int(dimensions [1])
    bits_per_pixel = file_size/pixels
    bpp_list.append(bits_per_pixel)
    log_string = 'Image:' + str(Path(file).stem) + '.bpg\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(file_size) + '\tWidth:' + str(dimensions[0]) + '\tHeight:' + str(dimensions[1]) + '\tPixels:' + str(pixels) + '\tCompressionTime:' + str(compression_times[count])
    log.writelines([log_string,'\n'])
    print(log_string)
    count +=1

log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')])
print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')
log.close()

Image:IMG0001.bpg	bpp:0.30695215861002606	Size:1931176	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.7602776050000557
Image:IMG0002.bpg	bpp:0.16990788777669272	Size:1068968	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.311306874000138
Image:IMG0003.bpg	bpp:0.06911595662434895	Size:434840	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.188663384000165
Image:IMG0004.bpg	bpp:0.13702774047851562	Size:862104	Width:2048	Height:3072	Pixels:6291456	CompressionTime:2.2395813810001073
Image:IMG0005.bpg	bpp:0.24157206217447916	Size:1519840	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.3182987949999188
Image:IMG0006.bpg	bpp:0.22128804524739584	Size:1392224	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.27435170800004
Image:IMG0007.bpg	bpp:0.08691151936848958	Size:546800	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.31635398100002
Image:IMG0008.bpg	bpp:0.3069864908854167	Size:1931392	Width:3072	Height:2048	Pixels:6291456	CompressionTime:3.180

# VVC Conversion
## Setting input and output directory for VVC conversion

In [64]:
source_path = str(current_path)+'/IMAGES/YUV_IMAGES'
source_path.replace(os.sep,ntpath.sep)
print("Source: ",source_path)

destination_path = str(current_path)+'/IMAGES/VVC_IMAGES'
destination_path.replace(os.sep,ntpath.sep)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

destination_path = str(current_path)+'/IMAGES/VVC_IMAGES/YUV'
destination_path.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

destination_path_VVC = str(current_path)+'/IMAGES/VVC_IMAGES/VVC'
destination_path_VVC.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path_VVC)
if os.path.isdir(destination_path_VVC) & os.path.exists(destination_path_VVC):
    try:
        files = os.listdir(destination_path_VVC)
        for file in files:
            file_path = os.path.join(destination_path_VVC, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path_VVC)

log_path = str(current_path)+'/IMAGES/VVC_IMAGES'
source_path.replace(os.sep,ntpath.sep)
print("Log: ",source_path)

Source:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES
Destination directory has been emptied
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES/YUV
Destination directory has been emptied
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES/VVC
Destination directory has been emptied
Log:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES


## Converting images to VVC

In [65]:
compression_times = []
quantizer_parameter = 35  # 32 = 0.25bpp ;  Quantization parameter (0...63) 0=better quality
for file in os.listdir(source_path):
    if file.lower().endswith('.yuv'):
        dimensions = open(source_path+'/'+str(Path(file).stem)+'.txt').readline()
        source_file = str(source_path + '/' + file)
        source_file.replace(os.sep,ntpath.sep)
        destination_file = str(destination_path_VVC + '/' + str(Path(file).stem) + '.vvc')
        destination_file.replace(os.sep,ntpath.sep)
        starttime = timeit.default_timer()
        subprocess.run([str(vvenc_path),'--profile','main_10_still_picture','-f','1','-c','yuv420_10','-s',str(dimensions),'--qp',str(quantizer_parameter),'--input',str(source_file),'--output',str(destination_file)])
        endtime = timeit.default_timer()
        execution_time = endtime - starttime - 0.2
        compression_times.append(execution_time)
        print('Encoded', Path(file).stem, ' | Execution time: ', str(execution_time), ' seconds','\n\n')

vvencapp: Fraunhofer VVC Encoder ver. 1.9.1 [Linux][GCC 13.2.1][64 bit][SIMD=AVX2]
vvenc [info]: Input File                             : /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES/IMG0001.yuv
vvenc [info]: Bitstream File                         : /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES/VVC/IMG0001.vvc
vvenc [info]: Real Format                            : 3072x2048  yuv420p10  60 Hz  SDR  1 frame
vvenc [info]: Frames                                 : encode 1 frame 
vvenc [info]: Internal format                        : 3072x2048  60 Hz  SDR
vvenc [info]: Threads                                : 8  (parallel frames: 4)
vvenc [info]: Rate control                           : QP 35
vvenc [info]: Perceptual optimization                : Enabled
vvenc [info]: Intra period (keyframe)                : 64
vvenc [info]: Decoding refresh type                  : CRA


vvenc [info]: stats summary: frame= 1/1 avg_fps= 0.1 avg_bitrate= 89597.76

## Converting VVC Images back to YUV for later comparison
To convert images back to PNG refer to ImagePreparation

In [67]:
for file in os.listdir(destination_path_VVC):
    source_file = str(destination_path_VVC + '/' + file)
    source_file.replace(os.sep,ntpath.sep)
    destination_file = str(destination_path+ '/' + str(Path(file).stem) + '.yuv')
    destination_file.replace(os.sep,ntpath.sep)
    starttime = timeit.default_timer()
    subprocess.run([str(vvdec_path),'--bitstream',str(source_file),'--output',str(destination_file)])
    endtime = timeit.default_timer()
    print('Decoded', Path(file).stem, ' | Execution time: ', endtime - starttime, ' seconds','\n\n')

Fraunhofer VVC/H.266 Decoder VVdeC, version 2.1.2 [THREADS=12; PARSE_DELAY=12; SIMD=AVX2]
POC    0 LId:  0 TId: 0 ( I-SLICE, QP 32 ) [DT  0.113] [L0 ] [L1 ] 
vvdecapp [info]: SizeInfo: 3072x2048 (10b)
vvdecapp [info]: 2023-Oct-22 19:28:06.051845: 1 frames decoded @ 13.5135 fps (0.074 sec)

Decoded IMG0001  | Execution time:  0.1251940169995578  seconds 


Fraunhofer VVC/H.266 Decoder VVdeC, version 2.1.2 [THREADS=12; PARSE_DELAY=12; SIMD=AVX2]
POC    0 LId:  0 TId: 0 ( I-SLICE, QP 30 ) [DT  0.127] [L0 ] [L1 ] 
vvdecapp [info]: SizeInfo: 3072x2048 (10b)
vvdecapp [info]: 2023-Oct-22 19:28:06.142759: 1 frames decoded @ 12.3457 fps (0.081 sec)

Decoded IMG0002  | Execution time:  0.0911351079994347  seconds 


Fraunhofer VVC/H.266 Decoder VVdeC, version 2.1.2 [THREADS=12; PARSE_DELAY=12; SIMD=AVX2]
POC    0 LId:  0 TId: 0 ( I-SLICE, QP 27 ) [DT  0.073] [L0 ] [L1 ] 
vvdecapp [info]: SizeInfo: 3072x2048 (10b)
vvdecapp [info]: 2023-Oct-22 19:28:06.206294: 1 frames decoded @ 18.5185 fps (0.054

## Compute bits per pixel of cf converted images

In [66]:
bpp_list=[]
count = 0
log = open(str(log_path)+'/log.txt', 'w')
log.writelines([str(datetime.now()),'\n'])
for file in os.listdir(destination_path_VVC):
    filepath = str(destination_path_VVC) + '/' + str(file)
    filepath.replace(os.sep,ntpath.sep)
    file_size = os.path.getsize(filepath) * 8
    dimensions = open(source_path+'/'+str(Path(file).stem)+'.txt').readline().split('x')
    pixels = int(dimensions[0]) * int(dimensions [1])
    bits_per_pixel = file_size/pixels
    bpp_list.append(bits_per_pixel)
    log_string = 'Image:' + str(Path(file).stem) + '.vvc\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(file_size) + '\tWidth:' + str(dimensions[0]) + '\tHeight:' + str(dimensions[1]) + '\tPixels:' + str(pixels) + '\tCompressionTime:' + str(compression_times[count])
    log.writelines([log_string,'\n'])
    print(log_string)
    count +=1

log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')])
print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')
log.close()

Image:IMG0001.vvc	bpp:0.2373530069986979	Size:1493296	Width:3072	Height:2048	Pixels:6291456	CompressionTime:10.752857676999884
Image:IMG0002.vvc	bpp:0.2120361328125	Size:1334016	Width:3072	Height:2048	Pixels:6291456	CompressionTime:11.9030521040002
Image:IMG0003.vvc	bpp:0.0825347900390625	Size:519264	Width:3072	Height:2048	Pixels:6291456	CompressionTime:7.398505088000365
Image:IMG0004.vvc	bpp:0.16753641764322916	Size:1054048	Width:2048	Height:3072	Pixels:6291456	CompressionTime:9.43093889000038
Image:IMG0005.vvc	bpp:0.20131937662760416	Size:1266592	Width:3072	Height:2048	Pixels:6291456	CompressionTime:12.196952049999982
Image:IMG0006.vvc	bpp:0.17770258585611978	Size:1118008	Width:3072	Height:2048	Pixels:6291456	CompressionTime:9.767770573000053
Image:IMG0007.vvc	bpp:0.09375762939453125	Size:589872	Width:3072	Height:2048	Pixels:6291456	CompressionTime:8.942849575999663
Image:IMG0008.vvc	bpp:0.2066218058268229	Size:1299952	Width:3072	Height:2048	Pixels:6291456	CompressionTime:15.48347712